# Mapping 3
- Line of Interest (circle & straight line)
- Frequency spectrum mapping
- Linear interpolation

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from datetime import datetime
from IPython.display import Video, Audio
from pya import *

import simulation as sim

%matplotlib widget

In [ ]:
plt.close()
plt.figure()
frame = sim.parabolic(center=dict(x=28, y=64), factor=dict(x=8, y=8))
plt.pcolormesh(np.abs(np.square(frame)), cmap='inferno', norm=matplotlib.colors.PowerNorm(vmin=0, vmax=np.max(np.abs(np.square(frame))), gamma=0.5))
# plt.pcolormesh(potential, vmin=0, vmax=20000)
plt.colorbar()
plt.show()

In [ ]:
s = startup()

## Simulation

In [ ]:
n = 128

multi_slit = [(-15, -13), (-8, -6), (-1, 1), (6, 8), (13, 15)]
double_slit = [(-7, -5), (6, 8)]
single_slit = [(-5, 6)]
no_slits = []

## Sonification

### Scanner functions
Return a scanner (function) for the sonification functions to use later.

The scanners recieve a timestamp and a normalized phase to return the simulation value on that at that timestamp and phase value.

In [ ]:
def line_of_interest_scanner(start_coordinates={'x': 0, 'y': 64}, end_coordinates={'x': 128, 'y': 64}):
    """Returns a function that scans the simulation along a line"""
    start_coordinates = np.array([start_coordinates['y'], start_coordinates['x']])[np.newaxis, :]
    end_coordinates = np.array([end_coordinates['y'], end_coordinates['x']])[np.newaxis, :]

    def line_of_interest(simulation: sim.Simulation, t, phase_0_to_1):
        phase_0_to_1 = phase_0_to_1[:, np.newaxis]
        coordinates = (1 - phase_0_to_1) * start_coordinates + phase_0_to_1 * end_coordinates

        indices = np.array([t, coordinates[:, 0], coordinates[:, 1]])
        return simulation.interpolated(indices)

    return line_of_interest


def circle_of_interest_scanner(center_coordinates={'x': 64, 'y': 64}, radius=32):
    """Returns a function that scans the simulation along a circle"""
    center_x = center_coordinates["x"]
    center_y = center_coordinates["y"]

    def circle_of_interest(simulation: sim.Simulation, t, phase_0_to_1):
        angle = 2*np.pi * phase_0_to_1
        x = center_x + radius * np.cos(angle)
        y = center_y + radius * np.sin(angle)
        
        indices = np.array([t, y, x])
        return simulation.interpolated(indices)
    
    return circle_of_interest

### Path of Interest - Audification
Scans the simulation along the above defined paths at the given frequency. The result is directly used as signal.

Analogous to slicing the simulation at this path and using the result as wavetable.

In [ ]:
def audification(simulation: sim.Simulation, frequency=100, sample_rate=44100, scanner_fn=line_of_interest_scanner(), complex_to_real_fn=sim.probability_density):
    num_samples = int(sample_rate * simulation.duration_seconds())
    t = np.linspace(0, simulation.duration_seconds() * simulation.fps, num_samples, endpoint=False)
    phase_0_to_1 = np.linspace(0, frequency * simulation.duration_seconds(), num_samples, endpoint=False) % 1

    simulation_values = scanner_fn(simulation, t, phase_0_to_1)
    signal = complex_to_real_fn(simulation_values)
    return Asig(signal, sr=sample_rate)

### Timbre mapping

In [ ]:
def timbre_mapping(simulation: sim.Simulation, num_harmonics=32, spacing='log', frequency=100, sample_rate=44100, scanner_fn=line_of_interest_scanner(), complex_to_real_fn=sim.probability_density):
    num_samples = int(sample_rate * simulation.duration_seconds())
    t = np.linspace(0, simulation.duration_seconds() * simulation.fps, num_samples, endpoint=False)

    partial_number = np.arange(num_harmonics) + 1

    if (spacing == 'lin'): 
        positions = np.linspace(0, 1, num_harmonics)
    if (spacing == 'log'):
        positions = np.log2(partial_number) / np.log2(num_harmonics)

    # make 2d arrays to have each timestep have one entry for each position
    t = np.repeat(t[:, np.newaxis], num_harmonics, axis=1)
    positions = np.repeat(positions[np.newaxis, :], num_samples, axis=0)

    # flatten as scanner takes 1d inputs and rebuild the shape again
    simulation_values = scanner_fn(simulation, t.flatten(), positions.flatten()).reshape(num_samples, num_harmonics)
    amplitudes = complex_to_real_fn(simulation_values) / partial_number[np.newaxis, :]

    frequencies = np.reshape(frequency * partial_number, (1, -1))
    phases = np.reshape(np.random.random((num_harmonics)), (1, -1)) # random phase offsets to impulse like waveshape 
    video_seconds = np.reshape(np.linspace(0, simulation.duration_seconds(), num_samples, endpoint=False), (-1, 1))

    signal = np.sum(amplitudes * np.cos(2*np.pi * (video_seconds * frequencies + phases)), axis=1)
    return Asig(signal, sr=sample_rate)

### Field Audification

In [ ]:
def field_audification(simulation: sim.Simulation, scanline='vertical', x_stride=1, y_stride=1, time_stride=50, complex_to_real_fn=sim.probability_density):
    height = simulation.frames.shape[1]
    width = simulation.frames.shape[2]
    frames = simulation.frames[::time_stride, ::y_stride, ::x_stride] # apply stride
    
    if scanline == 'horizontal': frames = frames
    elif scanline == 'vertical': frames = frames.transpose(0, 2, 1)
    else: print("User warning: Specified scanline is invalid. Defaulting to vertical.")
    
    signal = complex_to_real_fn(frames).flatten()
    sample_rate = int(np.round(simulation.fps/time_stride * height/y_stride * width/x_stride))
    return Asig(signal, sample_rate)

## Scenarios
Note: Running the simulation and rendering the video can take about 30 seconds on an average device. Some sonifications also take that long.

### Scenario 1: Harmonic Oscillator

In [ ]:
harmonic_oscillator = sim.Simulation(
    title = "Harmonic Oscillator - Parabolic Potential",
    fps = 200,
    speed = 0.004,
    initial_state = sim.gaussian(),
    potential = sim.parabolic(),
    barrier = None
)
harmonic_oscillator.simulate(seconds=8)

In [ ]:
Video(harmonic_oscillator.render_video(gamma=0.6))

In [ ]:
asig = audification(
    simulation=harmonic_oscillator, 
    frequency=100, 
    scanner_fn=circle_of_interest_scanner(center_coordinates=dict(x=64, y=64), radius=32), 
    complex_to_real_fn=sim.probability_density
)
Video(harmonic_oscillator.video_with_sonification(asig, sonification_title="audification"))

In [ ]:
asig = timbre_mapping(
    simulation=harmonic_oscillator,
    num_harmonics=20,
    spacing='log', # 'lin'
    frequency=100,
    scanner_fn=line_of_interest_scanner(start_coordinates=dict(x=10, y=64), end_coordinates=dict(x=118, y=64)),
    complex_to_real_fn=sim.probability_density
)
Video(harmonic_oscillator.video_with_sonification(asig, sonification_title="timbre_mapping"))

In [ ]:
sonification_fps=8
asig = field_audification(
    simulation=harmonic_oscillator,
    scanline='vertical',
    x_stride=4,
    y_stride=4,
    time_stride=int(harmonic_oscillator.fps / sonification_fps),
    complex_to_real_fn=sim.probability_density
)
Video(harmonic_oscillator.video_with_sonification(asig, sonification_title="field_audification"))

### Scenario 2: Tunnel Effect

In [ ]:
tunnel_effect = sim.Simulation(
    title = "Tunnel Effect - Barrier and Parabolic Potential",
    fps = 230,
    speed = 0.004,
    initial_state = sim.gaussian(),
    potential = sim.parabolic(),
    barrier = sim.Barrier(x=64, width=1, slits=[])
)
tunnel_effect.simulate(seconds=10)

In [ ]:
Video(tunnel_effect.render_video(gamma=0.6))

### Scenario 3: Single Slit

In [ ]:
single_slit = sim.Simulation(
    title = "Slingle Slit and Parabolic Potential",
    fps = 240,
    speed = 0.002,
    initial_state = sim.gaussian(),
    potential = sim.parabolic(),
    barrier = sim.Barrier(x=64, width=1, slits=[[64-5, 64+5]])
)
single_slit.simulate(seconds=10)

In [ ]:
Video(single_slit.render_video(gamma=0.3))

### Scenarion 4: Double Slit

In [ ]:
double_slit = sim.Simulation(
    title = "Double Slit and Parabolic Potential",
    fps = 240,
    speed = 0.002,
    initial_state = sim.gaussian(),
    potential = sim.parabolic(),
    barrier = sim.Barrier(x=64, width=1, slits=[[64-7, 64-5], [64+5, 64+7]])
)
double_slit.simulate(seconds=10)

In [ ]:
Video(double_slit.render_video(gamma=0.3))